In [2]:
import json,re,string,nltk
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.cross_validation import train_test_split

In [3]:
train_json= pd.read_json('train.json')
test_json= pd.read_json('test.json')

#we only care about the cuisine and the list of ingredients
train_labels_all=train_json['cuisine']
train_data_all=train_json['ingredients']
test_data= test_json['ingredients']

#join all the ingredients in one long string separated by comma
train_data_all=["|".join(row) for row in train_data_all]
test_data=["|".join(row) for row in test_data]

# split train data into train and dev set
train_data, dev_data, train_labels, dev_labels = train_test_split( train_data_all, train_labels_all, test_size=0.2, random_state=42)


In [28]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
#stemmer = PorterStemmer()
#stemmer=SnowballStemmer("english")
wnl = WordNetLemmatizer()

def pp1(text):
    results=""
    for i,ingred in enumerate(text.split("|")):
        clean_ingred=""
        tokens = nltk.word_tokenize(ingred)# tokenize text and lemmatize
        tokens= [wnl.lemmatize(t) for t in tokens]
        #tokens= [stemmer.stem(t) for t in tokens]
        stopWords = ['and','inch','cup','spoon','can','bottle','thick','oz','jar','half','bottle','kilo','kg','gram','liter','lbs','once']
        words = []
        for items in tokens[-2:]:
            if items.lower() not in [wnl.lemmatize(w) for w in stopWords] :
                items = re.sub("[^\w ]", "", items) # remove non words
                items=re.sub(r'\b\w?\d+\w?\b',' ',items) # remove numbers
                words.extend([items.lower()])
        clean_ingred=" ".join(words).strip(" ")
        if i==0:
            results=results+clean_ingred
        else:
            results=results+"|"+clean_ingred #reassemble the cleaned ingredients back to a string
    return results


#use tfidf vectorizer to tokenize and output a matrix that maps recipes along of the ingredients
vectorizer = TfidfVectorizer(strip_accents='unicode',ngram_range=(1,1),token_pattern='\|*([^\|]+)\|*',min_df=1,preprocessor=pp1)
vectorizer.fit_transform(train_data_all)



<39774x5004 sparse matrix of type '<type 'numpy.float64'>'
	with 426259 stored elements in Compressed Sparse Row format>

In [129]:
vectorizer.get_feature_names()

[u'aai',
 u'aai powder',
 u'abalon',
 u'abura age',
 u'acai pure',
 u'accent',
 u'accent season',
 u'achiot',
 u'achiot past',
 u'achiot powder',
 u'acke',
 u'acorn squash',
 u'adobo',
 u'adobo sauc',
 u'adobo season',
 u'adzuki bean',
 u'agar',
 u'agav nectar',
 u'ahi',
 u'aioli',
 u'ajinomoto',
 u'ajwain',
 u'aka miso',
 u'albacor',
 u'alcohol',
 u'ale',
 u'aleppo pepper',
 u'alfalfa sprout',
 u'alfredo',
 u'alfredo sauc',
 u'allpurpos flour',
 u'allspic',
 u'allspic berri',
 u'almond',
 u'almond butter',
 u'almond extract',
 u'almond fill',
 u'almond flour',
 u'almond liqueur',
 u'almond meal',
 u'almond milk',
 u'almond oil',
 u'almond past',
 u'alphabet pasta',
 u'alum',
 u'amaranth',
 u'amaretti',
 u'amaretti cooki',
 u'amaretto',
 u'amaretto liqueur',
 u'amber',
 u'amchur',
 u'american chees',
 u'ampalaya',
 u'anaheim chile',
 u'anasazi bean',
 u'ancho',
 u'ancho powder',
 u'anchovi',
 u'anchovi filet',
 u'anchovi fillet',
 u'anchovi past',
 u'andouill sausag',
 u'angel hair',
 

In [29]:
Xtrain= vectorizer.transform(train_data)
Xdev= vectorizer.transform(dev_data)

In [ ]:
# Use Gradient Boosting Classifier
#clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1,max_depth=1, random_state=0)
cvectorizer = CountVectorizer(min_df=1, max_features=2000)
cvectorizer.fit_transform(train_data_all)
Xtrain= cvectorizer.transform(train_data)
Xdev= cvectorizer.transform(dev_data)
clf = GradientBoostingClassifier(n_estimators=50, verbose=2)
clf.fit(Xtrain.toarray(),train_labels)

print clf.score(Xdev.toarray(),dev_labels)


      Iter       Train Loss   Remaining Time 
         1       75597.9376          335.91m
         2       69630.8516         1804.42m
         3       64975.9882         1298.31m
         4       61601.3251         1040.27m
         5       58508.7436          879.76m
         6       56063.9925          895.57m
         7       54027.9616          803.05m

In [25]:
#####       BASELINE       #####

# Use Logistic regression with count vectorizer 
#use count vectorizer to tokenize and output a matrix that maps recipes along of the ingredients
#cvectorizer = CountVectorizer(strip_accents='unicode',ngram_range=(1,1),token_pattern='\|*([^\|]+)\|*', preprocessor=pp1, max_features=2000)
cvectorizer = CountVectorizer(min_df=1, max_features=2000)

cvectorizer.fit_transform(train_data_all)
Xtrain= cvectorizer.transform(train_data)
Xdev= cvectorizer.transform(dev_data)
clf = LogisticRegression(C=10)
clf.fit(Xtrain.toarray(),train_labels)
print clf.score(Xdev.toarray(),dev_labels)
#print cvectorizer.get_feature_names()
###0.776869893149

[u'10', u'14', u'33', u'40', u'abura', u'acai', u'accent', u'achiote', u'acid', u'ackee', u'acorn', u'acting', u'active', u'added', u'adobo', u'adzuki', u'agar', u'agave', u'aged', u'ahi', u'aioli', u'ajwain', u'albacore', u'ale', u'aleppo', u'alfalfa', u'alfredo', u'all', u'allspice', u'almond', u'almonds', u'amaretti', u'amaretto', u'amchur', u'american', u'aminos', u'anaheim', u'ancho', u'anchovies', u'anchovy', u'and', u'andouille', u'anejo', u'angel', u'anglaise', u'angostura', u'anise', u'anjou', u'annatto', u'apple', u'apples', u'applesauce', u'applewood', u'apricot', u'apricots', u'arbol', u'arborio', u'arhar', u'armagnac', u'arrowroot', u'artichok', u'artichoke', u'artichokes', u'artisan', u'arugula', u'asadero', u'asafetida', u'asafoetida', u'asiago', u'asian', u'asparagus', u'assorted', u'atta', u'au', u'avocado', u'avocados', u'a\xe7ai', u'baby', u'bacardi', u'back', u'bacon', u'bags', u'baguette', u'baileys', u'baked', u'baking', u'balls', u'balm', u'balsamic', u'balsamico

In [30]:
# Use Logistic regression with tfidf vectorizer

clf = LogisticRegression(C=10)
clf.fit(Xtrain.toarray(),train_labels)
print clf.score(Xdev.toarray(),dev_labels)
###Best score so far: 0.789440603394
## 2000 features=0.783406662476
## 3000 features= 0.78667504714
## 4000 features= 0.776869893149
## all 5004 features = 0.789566310497

0.789566310497


In [ ]:
# Use Voting classifier with Grid Search
clf1 = LogisticRegression(C=10)
clf2 = RandomForestClassifier(random_state=1)
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2)], voting='soft')
params = {'lr__C': [1.0, 10.0], 'rf__n_estimators': [20, 200]}
clf = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
clf = clf.fit(Xtrain.toarray(),train_labels)
print clf.score(Xdev.toarray(),dev_labels)


In [141]:
# Generate predictions for submission
Xtest=vectorizer.transform(test_data)
Ytest=clf.predict(Xtest.toarray())
Ytest

results=pd.DataFrame()
results['id']=test_json['id']
results['cuisine']=Ytest

results.to_csv('submission.csv',delimiter=',',index=False)